In [19]:
import numpy as np
import matplotlib.pyplot as plt
import os, glob
import pickle
import pandas as pd
import cv2
%matplotlib inline

# classification required packages
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report
import pickle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [20]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [21]:
def load_data(filedir, filepath, csvfile):
    data = np.load(os.path.join(filedir, filepath), allow_pickle=True)
    train_info = pd.read_csv(os.path.join(filedir, csvfile))
    labels = np.array(train_info['ClassId'])
    
    return data, labels

def resample(data, labels, n=100):
    resample_data = []
    resample_labels = []
    for i in range(43):
        index = np.where(labels == i)[0][:n]
        resample_data.append(data[index])
        resample_labels.append(labels[index])
    return np.array(resample_data).reshape(43*n, -1), np.array(resample_labels).reshape(43*n, -1)

def randomize_data(data, labels):
    randomize = np.arange(len(labels))
    np.random.shuffle(randomize)
    X = data[randomize]
    y = labels[randomize]
    
    return X,y

def resize_img(images):
    ret = []
    for im in images:
        im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (32, 32))
        #print(im.shape)
        ret.append(im)
    ret = np.array(ret)
    #print(ret.shape)
    return ret

In [22]:
data_dir = '/content/drive/MyDrive/GTSRB'
ori_train_path = 'Original/trainImages.npy'
ori_test_path = 'Original/testImages.npy'

trainOriData, trainOriLabels = load_data(data_dir, ori_train_path, 'Train.csv')
trainOriData = resize_img(trainOriData)
trainOriData, trainOriLabels = randomize_data(trainOriData, trainOriLabels)
print(trainOriData.shape)

testOriData, testOriLabels = load_data(data_dir, ori_test_path, 'Test.csv')
testOriData = resize_img(testOriData)
print(testOriData.shape)

(39209, 32, 32, 3)
(12630, 32, 32, 3)


In [29]:
def train_model(estimators, trainData, trainLabels, testData, testLabels):
    for n in estimators:
        print(f'RANDOM FOREST WITH {n} ESTIMATORS')
        if os.path.isfile("rf_"+str(n)+"_ori_nofe_balanced.pkl"):
            print("[INFO] loading classifier: Random Forest trained on ori images...")
            rf = pickle.load(open("rf_"+str(n)+"_ori_nofe_balanced.pkl", 'rb'))
            print("[INFO] Classifer is loaded as instance ::rf::")
        else:
            print("[INFO] pre-trained classifier not found. \n Training Classifier Random Forest")
            rf = RandomForestClassifier(n_estimators=n, class_weight='balanced')
            rf.fit(trainData.reshape(len(trainLabels), -1),trainLabels)
            print("[INFO] Succefully trained the classsifier. \n Saving the classifier for further use")
            pickle.dump(rf, open("rf_"+str(n)+"_ori_nofe_balanced.pkl", 'wb')) 
            print("[INFO] Classifier Saved")
            
        predictions = rf.predict(testData.reshape(len(testLabels), -1))
 
        # show a final classification report demonstrating the accuracy of the classifier
        print("EVALUATION ON TESTING DATA FOR" + str(n) + 'FOREST')
        print(classification_report(testLabels, predictions))
        
        print('Accuracy of model on training data')

        print(rf.score(trainData.reshape(len(trainLabels), -1), trainLabels))
        print('-------------------------------------------------------')

In [30]:
estimators = [50,100,150,200]
train_model(estimators, trainOriData, trainOriLabels, testOriData, testOriLabels)

RANDOM FOREST WITH 50 ESTIMATORS
[INFO] loading classifier: Random Forest trained on ori images...
[INFO] Classifer is loaded as instance ::rf::
EVALUATION ON TESTING DATA FOR50FOREST
              precision    recall  f1-score   support

           0       0.50      0.03      0.06        60
           1       0.59      0.81      0.68       720
           2       0.55      0.66      0.60       750
           3       0.53      0.62      0.57       450
           4       0.61      0.73      0.66       660
           5       0.49      0.53      0.51       630
           6       0.63      0.51      0.56       150
           7       0.67      0.67      0.67       450
           8       0.69      0.48      0.57       450
           9       0.95      0.82      0.88       480
          10       0.89      0.93      0.91       660
          11       0.81      0.92      0.87       420
          12       0.93      0.91      0.92       690
          13       0.92      1.00      0.96       720
     